In [1]:
# Import dependencies
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sys
sys.path.insert(0, '../../')
from config import localHost, localPass, DATABASE_URL

In [4]:
# Connect to hosted postgres and test connection
engine = create_engine(DATABASE_URL)
data = engine.execute("SELECT * FROM pet_id")
for row in data:
    print(row)

(1, 'dog')
(2, 'cat')
(3, 'bird')
(4, 'fish')


In [23]:
# Query for all pet population data
q = "SELECT * FROM pet_population\
     NATURAL JOIN country_id\
     NATURAL JOIN pet_id\
     NATURAL JOIN happiness_data\
     INNER JOIN world_bank_2017 ON pet_population.country_id = world_bank_2017.country_id"

# Send query, clean response
petData = pd.read_sql(q, engine)
petData = petData.drop(["country_id", "pet_id"], axis = 1)

# Create dfs by pet type
dogData = petData.loc[petData['pet_type'] == "dog"]
catData = petData.loc[petData['pet_type'] == "cat"]
fishData = petData.loc[petData['pet_type'] == "fish"]
birdData = petData.loc[petData['pet_type'] == "bird"]

# Transform to dictionary
def makedictionary(df):
    queryDict = {}
    for col in list(df.columns.values):
        queryDict[col] = [x for x in df[col]]
    return queryDict
petDict = {}
petDict['dog'] = makedictionary(dogData)
petDict['cat'] = makedictionary(catData)
petDict['fish'] = makedictionary(fishData)
petDict['bird'] = makedictionary(birdData)

In [4]:
# Query for all world happiness data
q = "SELECT * FROM happiness_data\
     NATURAL JOIN country_id\
     INNER JOIN world_bank_2017 ON happiness_data.country_id = world_bank_2017.country_id"

# Send query, clean response
worldData = pd.read_sql(q, engine)
worldData = worldData.fillna(0)
worldData = worldData.drop("country_id", axis = 1)

# Transform to dictionary
worldDict = {}
for col in list(worldData.columns.values):
    worldDict[col] = [x for x in worldData[col]]

In [6]:
worldDict['per_capita_gdp']

[20815300220.0,
 124209000000.0,
 13039352744.0,
 382575000000.0,
 637590000000.0,
 11536590636.0,
 1323420000000.0,
 416596000000.0,
 40747792238.0,
 3477502178.0,
 492681000000.0,
 9273558272.0,
 12873114800.0,
 249724000000.0,
 56831518294.0,
 35307127660.0,
 18168579578.0,
 54442374373.0,
 1838000000.0,
 37508642113.0,
 2055510000000.0,
 2511852941.0,
 17406530781.0,
 1949411659.0,
 1653040000000.0,
 678887000000.0,
 277076000000.0,
 12237700000000.0,
 40388624117.0,
 34798596482.0,
 37241300949.0,
 8722553001.0,
 309191000000.0,
 57057372468.0,
 21651791751.0,
 21651791751.0,
 215726000000.0,
 3677440000000.0,
 324872000000.0,
 75931656815.0,
 170371000000.0,
 103057000000.0,
 235369000000.0,
 235369000000.0,
 1311320000000.0,
 25921079612.0,
 80561496134.0,
 251885000000.0,
 2582500000000.0,
 14622880886.0,
 2622430000000.0,
 15159281211.0,
 47330016343.0,
 10496056733.0,
 200288000000.0,
 75620095538.0,
 341449000000.0,
 22978532897.0,
 54849180229.0,
 8408150518.0,
 13913500000

In [9]:
import jsonify

In [11]:
jsonify(worldDict)

TypeError: 'module' object is not callable

In [ ]:
type(worldDict)